In [1]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import silhouette_score, silhouette_samples
#import matplotlib.ticker as ticker
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
#import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.stats import norm
import itertools
import networkx as nx
import plotly.figure_factory as ff
from itertools import combinations
from IPython.display import display
from pacmap import PaCMAP
from scipy.cluster.hierarchy import linkage, fcluster

In [2]:
cancer_type = "Brain"
non_coding_region = 'acceptor'
file_name = "df_log_odd_score_patientwise.tsv"

In [3]:
file_path= '/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/Variant_Analysis/{}/{}'.format(cancer_type, non_coding_region, file_name)
output_path = "/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/Kmeans_cluster/Categorise/"
output_subset_path = "/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/Kmeans_cluster/Categorise/Subset"
# Subfolders within the base path

In [4]:
df = pd.read_csv(file_path, sep='\t')
df = df.set_index('Patient_Core_ID', drop=True)
df

,chr1_11988386_11988465_11988402_11988403_G_GTT,chr1_16041210_16041289_16041214_16041215_G_GCATGAGC,chr1_17334283_17334362_17334306_17334307_A_AT,chr1_17334283_17334362_17334306_17334307_A_ATT,chr1_47225850_47225929_47225895_47225900_GCAGA_G,chr1_54204977_54205056_54204984_54204988_CCTT_C,chr1_54227121_54227200_54227180_54227181_A_AGG,chr1_54977823_54977902_54977864_54977865_G_GAAA,chr1_59021086_59021165_59021103_59021104_A_AT,chr1_66612016_66612095_66612025_66612029_GAGA_G,...,chr8_126848479_126848558_126848485_126848488_CAA_C,chr9_15888942_15889021_15888948_15888952_CTTT_C,chr9_77205272_77205351_77205285_77205287_CT_C,chr9_83855015_83855094_83855038_83855041_CTT_C,chrX_118447019_118447098_118447031_118447034_ATT_A,chr15_50930764_50930843_50930781_50930782_G_GT,chr6_42619427_42619506_42619448_42619457_ATATATTTT_A,chr16_35244229_35244308_35244269_35244270_G_C,chr22_24627811_24627890_24627822_24627823_G_A,chr6_32522135_32522214_32522156_32522157_C_A
Patient_Core_ID,,,,,,,,,,,,,,,,,,,,,
02eb77ff-2e07-4d56-9c32-d9fa415c7ce2,-3.351219,15.646363,NaN,-5.739093,NaN,NaN,NaN,NaN,NaN,24.229843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
033db366-1af1-4bd9-8339-c54800b1de57,NaN,15.646363,NaN,-5.739093,NaN,NaN,NaN,6.535246,NaN,24.229843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03777cdf-6779-4d23-93af-5b74fb982d7b,NaN,15.646363,NaN,-5.739093,-5.860069,NaN,-1.828692,6.535246,NaN,24.229843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
038750e2-9994-4420-bfa9-a3756683ddd1,NaN,15.646363,NaN,-5.739093,-5.860069,NaN,NaN,6.535246,21.681342,24.229843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
03ba06a3-5dc6-4dad-84ed-7fe3196d7402,NaN,NaN,NaN,-5.739093,NaN,NaN,NaN,NaN,NaN,24.229843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f8b8e0df-4234-4839-b0ee-34a2c9ef4985,NaN,15.646363,NaN,-5.739093,NaN,NaN,NaN,6.535246,21.681342,24.229843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f90f1702-d5f1-43f4-b167-2c440f941bf6,NaN,15.646363,NaN,-5.739093,-5.860069,NaN,-1.828692,6.535246,21.681342,24.229843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f9554b80-de24-4930-b651-7b2ae3b4ba8c,NaN,15.646363,NaN,-5.739093,-5.860069,NaN,-1.828692,NaN,21.681342,24.229843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Define a function to categorize based on the overall mean and standard deviation
def categorize_value(x, overall_mean, overall_std):
    if x == 0:
        return 0
    elif x > overall_mean + overall_std:
        return 1
    elif x < overall_mean - overall_std:
        return -1
    else:
        return 0

In [6]:
# Custom function to calculate Mean Absolute Deviation
def calculate_mad(column):
    mean = column.mean()
    return (column - mean).abs().mean()

In [7]:
def silhouette_plot(k, cluster_labels, silhouette_avg, sample_silhouette_values, folder_path, top_n_percent):
    fig = go.Figure()
    y_lower = 10  # Initial lower margin for the silhouette plot
    cluster_info = []  # To store info about clusters for the legend

    for i in range(k):
        # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        cluster_info.append(f"Cluster {i} [{size_cluster_i} patients]")  # Append cluster info for legend

        fig.add_trace(go.Scatter(
            x=ith_cluster_silhouette_values,
            y=np.arange(y_lower, y_upper),
            mode='lines',
            line=dict(width=0.5),
            fill='tozerox',
            name=f'Cluster {i} [{size_cluster_i} patients]'
        ))

        y_lower = y_upper + 10  # Gap between silhouettes

    # Red dashed line for average silhouette score
    fig.add_trace(go.Scatter(
        x=[silhouette_avg, silhouette_avg],
        y=[0, y_lower],
        mode='lines',
        line=dict(color="red", dash='dash', width=2),
        name='Average silhouette score'
    ))

    # Update plot layout
    fig.update_layout(
        title=f'Silhouette Analysis for k={k} Clusters for top {top_n_percent}% features <br>Average silhouette score {silhouette_avg:.4f}',
        xaxis_title='Silhouette Coefficient Values',
        yaxis_title='Cluster Label',
        yaxis=dict(range=[-0.1, y_lower + 10], showticklabels=False),
        showlegend=True
    )

    # Save and display the figure
    fig.write_html(f"{folder_path}/silhouette_plot_k_{k}_top_feature_{top_n_percent}.html")
    #fig.show()

In [8]:
def kmeans_silhouette_score(scaled_data, n_clusters, result_path, original_indices, top_percentage):
    # Initialize and run KMeans clustering with varying number of clusters
    #display(scaled_data)
    kmeans = KMeans(n_clusters=n_clusters, max_iter=300, tol=1e-04, init='k-means++', n_init=10, random_state=42)
    labels = kmeans.fit_predict(scaled_data)
    # Calculate and return the silhouette score
    score = silhouette_score(scaled_data, labels)
    sample_silhouette_values = silhouette_samples(scaled_data, labels)
    silhouette_plot(k,labels, score, sample_silhouette_values, result_path, top_percentage)
    

    # Create text files for each cluster
    for i in range(n_clusters):
        cluster_file_path = f"{result_path}/cluster_{i}.txt"
        with open(cluster_file_path, 'w') as file:
            indices = original_indices[labels == i]
            for idx in indices:
                file.write(str(idx) + '\n')
    return score

In [9]:
def data_categorized_mad(data):
    # Assuming 'df' is your DataFrame
    # Calculate the mean for each numeric column, ignoring NaN values
    means = data.select_dtypes(include=[np.number]).mean()
    
    # Compute overall mean and standard deviation of these means
    overall_mean = means.mean()
    overall_std = means.std()
    
    # Replace NaN values with 0
    data.fillna(0, inplace=True)
    
    # Apply the categorization function to each element of the dataframe
    for col in data.select_dtypes(include=[np.number]).columns:
        data[col] = data[col].apply(lambda x: categorize_value(x, overall_mean, overall_std))
    #display(data)
    # Calculate MAD for each numeric column
    mad_values = {col: calculate_mad(data[col]) for col in data.select_dtypes(include=[np.number]).columns}

    # Sort columns by MAD in descending order
    mad_sorted = sorted(mad_values.items(), key=lambda x: x[1], reverse=True)
    return (data, mad_values, mad_sorted)

In [10]:
data_processed, mad_values, mad_sorted = data_categorized_mad(df)

In [11]:
num_columns = int(len(mad_sorted) * 0.25)
top_columns = [col[0] for col in mad_sorted[:num_columns]]

In [12]:
num_columns

842

In [13]:
data_processed[top_columns]

,chr19_32826642_32826721_32826677_32826680_AAC_A,chr22_47688555_47688634_47688570_47688573_CTT_C,chr17_30786686_30786765_30786696_30786697_A_AATCT,chr14_73927066_73927145_73927088_73927089_A_AT,chr20_49089206_49089285_49089225_49089228_GTT_G,chr14_105625340_105625419_105625405_105625407_CA_C,chr11_33040089_33040168_33040109_33040110_C_CTTT,chr12_78808236_78808315_78808252_78808256_TTTG_T,chr14_36235283_36235362_36235302_36235303_C_CTTTTT,chr7_118322814_118322893_118322814_118322815_T_TTTTG,...,chr15_73697555_73697634_73697566_73697567_A_ATTTT,chr17_33634696_33634775_33634705_33634712_CTTTCTT_C,chr19_23134039_23134118_23134054_23134059_ATGTG_A,chr2_73610396_73610475_73610415_73610417_TC_T,chr2_218200967_218201046_218200990_218200991_A_AG,chr3_42834559_42834638_42834559_42834560_T_TTTAA,chr3_132665943_132666022_132665959_132665962_CAT_C,chr9_40993212_40993291_40993232_40993236_GCTT_G,chrX_119786693_119786772_119786750_119786753_CAA_C,chr1_20070253_20070332_20070290_20070291_C_A
Patient_Core_ID,,,,,,,,,,,,,,,,,,,,,
02eb77ff-2e07-4d56-9c32-d9fa415c7ce2,1,1,-1,0,0,-1,0,1,-1,-1,...,0,0,0,0,0,0,0,0,0,0
033db366-1af1-4bd9-8339-c54800b1de57,1,0,0,0,1,0,-1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03777cdf-6779-4d23-93af-5b74fb982d7b,1,1,0,-1,0,-1,0,1,-1,0,...,0,0,0,0,0,0,0,0,0,0
038750e2-9994-4420-bfa9-a3756683ddd1,0,0,-1,0,0,-1,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
03ba06a3-5dc6-4dad-84ed-7fe3196d7402,0,0,-1,0,1,0,-1,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
f8b8e0df-4234-4839-b0ee-34a2c9ef4985,1,1,0,0,0,-1,-1,1,-1,0,...,0,0,0,0,0,0,0,0,0,0
f90f1702-d5f1-43f4-b167-2c440f941bf6,0,0,-1,0,1,0,-1,1,0,-1,...,0,0,0,0,0,0,0,0,0,0
f9554b80-de24-4930-b651-7b2ae3b4ba8c,0,0,-1,0,1,-1,-1,0,-1,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
def save_cluster_results(cluster_1, cluster_2, iteration):
    # Create a folder for the iteration if it doesn't exist
    directory = f'/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/Kmeans_cluster/Categorise_stepwise_top_0.25_K=2/Iteration_{iteration}'
    if not os.path.exists(directory):
        os.makedirs(directory)
    # Save patient IDs for each cluster
    pd.DataFrame(cluster_1.index).to_csv(f'{directory}/cluster_1_ids.csv')  # Index is saved by default
    pd.DataFrame(cluster_2.index).to_csv(f'{directory}/cluster_2_ids.csv')  # Index is saved by default

In [30]:
# Initialize
iteration = 0
current_data = data_processed[top_columns].copy()  # Assume top_columns does not include 'Patient_Core_ID'

while True:
    iteration += 1
    print(f"Iteration {iteration}")

    # Apply KMeans clustering
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(current_data)
    kmeans = KMeans(n_clusters=2, max_iter=300, tol=1e-04, init='k-means++', n_init=10, random_state=42)
    labels = kmeans.fit_predict(scaled_features)

    # Calculate silhouette score
    if len(set(labels)) > 1:  # Silhouette score requires at least 2 clusters
        score = silhouette_score(scaled_features, labels)
        print(f"Silhouette Score for iteration {iteration}: {score}")
    
    # Divide data into clusters using labels
    current_data['Cluster'] = labels
    cluster_1 = current_data[current_data['Cluster'] == 0]
    cluster_2 = current_data[current_data['Cluster'] == 1]
    print(len(cluster_1), len(cluster_2))

    # Save results
    save_cluster_results(cluster_1.drop(columns=['Cluster']), cluster_2.drop(columns=['Cluster']), iteration)
    
    # Determine the smallest cluster and check the size condition
    smallest_cluster = cluster_1 if len(cluster_1) < len(cluster_2) else cluster_2
    largest_cluster = cluster_2 if len(cluster_1) < len(cluster_2) else cluster_1

    if len(smallest_cluster) < 0.4 * len(current_data):
        # Remove the smallest cluster's data and continue with the largest cluster
        current_data = largest_cluster.drop(columns=['Cluster'])
        print("Continuing with the largest cluster for next iteration.")
    else:
        print("Stopping condition met.")
        break
    
    if len(current_data) == 0:
        print("No data left to cluster.")
        break

Iteration 1
Silhouette Score for iteration 1: 0.6046102931611251
2 214
Continuing with the largest cluster for next iteration.
Iteration 2
Silhouette Score for iteration 2: 0.5027126771837074
213 1
Continuing with the largest cluster for next iteration.
Iteration 3
Silhouette Score for iteration 3: 0.35050405944133345
10 203
Continuing with the largest cluster for next iteration.
Iteration 4
Silhouette Score for iteration 4: 0.014170652213796951
53 150
Continuing with the largest cluster for next iteration.
Iteration 5
Silhouette Score for iteration 5: 0.28657929341073396
149 1
Continuing with the largest cluster for next iteration.
Iteration 6
Silhouette Score for iteration 6: 0.33356535742533056
148 1
Continuing with the largest cluster for next iteration.
Iteration 7
Silhouette Score for iteration 7: 0.23398848835917013
1 147
Continuing with the largest cluster for next iteration.
Iteration 8
Silhouette Score for iteration 8: 0.19911686609166948
1 146
Continuing with the largest clu